<a href="https://colab.research.google.com/github/LeeMooHeon/Final-Team10/blob/main/final_0(%ED%8F%89%EC%A0%90_%EA%B0%9C%EC%88%98%EB%A7%8C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [ ]:
from google.colab import drive
import pandas as pd

# 1. 구글 드라이브 마운트
drive.mount('/content/drive')

# 2. 데이터 불러오기
base_path = "/content/drive/MyDrive"

links = pd.read_csv(f"{base_path}/links.csv")
movies = pd.read_csv(f"{base_path}/movies.csv")
ratings = pd.read_csv(f"{base_path}/ratings.csv")
tags = pd.read_csv(f"{base_path}/tags.csv")

# 확인
print(links.shape, movies.shape, ratings.shape, tags.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(87585, 3) (87585, 3) (32000204, 4) (2000072, 4)
Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')


In [ ]:
print(links.columns, movies.columns, ratings.columns, tags.columns)

Index(['movieId', 'imdbId', 'tmdbId'], dtype='object') Index(['movieId', 'title', 'genres'], dtype='object') Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object') Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object')


In [ ]:
import pandas as pd

# ratings, tags의 timestamp를 datetime으로 변환
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
tags['timestamp'] = pd.to_datetime(tags['timestamp'], unit='s')

# 2015년 1월 1일 이후 데이터만 추출
cutoff_date = pd.Timestamp('2015-01-01')
ratings = ratings[ratings['timestamp'] >= cutoff_date].copy()
tags = tags[tags['timestamp'] >= cutoff_date].copy()

# movies, links도 ratings 기준으로 필터링
movie_ids = ratings['movieId'].unique()
movies = movies[movies['movieId'].isin(movie_ids)].copy()
links = links[links['movieId'].isin(movie_ids)].copy()

# 마지막 날짜 확인
last_rating_date = ratings['timestamp'].max()
last_tag_date = tags['timestamp'].max()

# 확인
print("ratings shape:", ratings.shape)
print("tags shape:", tags.shape)
print("movies shape:", movies.shape)
print("links shape:", links.shape)
print("ratings 마지막 날짜:", last_rating_date)
print("tags 마지막 날짜:", last_tag_date)


ratings shape: (12902312, 4)
tags shape: (1652355, 4)
movies shape: (83780, 3)
links shape: (83780, 3)
ratings 마지막 날짜: 2023-10-13 02:29:07
tags 마지막 날짜: 2023-10-12 23:56:23


In [ ]:
import pandas as pd

# 사용자별 집계
user_stats = ratings.groupby("userId").agg(
    rating_count=("rating", "count"),
    avg_rating=("rating", "mean")
).reset_index()

# 라벨 생성 (평균보다 낮으면 0, 높으면 1)
threshold = user_stats["avg_rating"].mean()
user_stats["label"] = (user_stats["avg_rating"] > threshold).astype(int)

print(user_stats.head())


   userId  rating_count  avg_rating  label
0       8            31    4.322581      1
1      10            33    3.181818      0
2      16           295    2.691525      0
3      20           140    3.846429      0
4      22            63    3.722222      0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

# 입력(X), 출력(y)
X = user_stats[["rating_count"]]  # 여기서는 단순히 평점 개수만
y = user_stats["label"]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# RandomForest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("=== RandomForest ===")
print(classification_report(y_test, y_pred_rf))

# XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

print("=== XGBoost ===")
print(classification_report(y_test, y_pred_xgb))


=== RandomForest ===
              precision    recall  f1-score   support

           0       0.68      0.46      0.55     10137
           1       0.63      0.81      0.71     11613

    accuracy                           0.65     21750
   macro avg       0.65      0.63      0.63     21750
weighted avg       0.65      0.65      0.63     21750

=== XGBoost ===
              precision    recall  f1-score   support

           0       0.67      0.48      0.56     10137
           1       0.64      0.80      0.71     11613

    accuracy                           0.65     21750
   macro avg       0.66      0.64      0.63     21750
weighted avg       0.65      0.65      0.64     21750



/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [00:36:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
